In [1]:
print "hello"

hello


In [2]:
import sys
print "\n".join(sys.path)


C:\Python27\Lib\naoqi\lib
C:\Windows\SYSTEM32\python27.zip
C:\Python27\DLLs
C:\Python27\lib
C:\Python27\lib\plat-win
C:\Python27\lib\lib-tk
C:\Python27
C:\Python27\lib\site-packages
C:\Python27\lib\site-packages\opencv-4.5.4-py2.7.egg
C:\Python27\lib\site-packages\win32
C:\Python27\lib\site-packages\win32\lib
C:\Python27\lib\site-packages\Pythonwin
C:\Python27\lib\site-packages\IPython\extensions
C:\Users\manal\.ipython


In [3]:
import os
import sys

sys.path.append( 'C:\\Python27\\Lib\\naoqi\\lib' )


In [4]:
import qi

In [5]:
# Filename: pepper_interact.py (Python 2)

import os
import time
from naoqi import ALProxy

#IP = "10.0.1.40" # robot router
IP = "10.0.1.153" # when pepper is connected wired to the router
#IP = "10.2.233.93" #  herts.ac.uk
#IP = "172.20.10.2" # My hotspot

port="9559"
connection_url = "tcp://"+IP+":9559"

portInt = 9559
PepperHost = IP

# Initialize Pepper
tts = ALProxy("ALTextToSpeech", IP, 9559)

tts.setLanguage("English")


In [6]:
import qi
import argparse
import sys
import os
import time

class TTSModule(object):
    def __init__(self, app, qilog):
        super(TTSModule, self).__init__()
        app.start()
        session = app.session
        
        self.qilog = qilog
        session.waitForService("ALRobotPosture")
        session.waitForService("ALAnimatedSpeech")
        self.posture = session.service("ALRobotPosture")
        self.speech = session.service("ALAnimatedSpeech")
        
    def run(self, text):
        try:
            if text is None:
                while True:
                    qilog.info("Waiting for input...")
                    sys.stdout.write("> ")
                    tts = sys.stdin.readline()
                    qilog.info("Uttering '%s'..." % tts.rstrip("\n\r"))
                    self.speech.say(tts)
                    self.posture.goToPosture("Stand", 0.8)
            else:
                qilog.info("Uttering '%s'..." % text)
                self.speech.say(text)
                qilog.info("Finished.")
        except KeyboardInterrupt:
            print "Interrupted by user, stopping gracefully."
        except Exception, e:
            qilog.error("Error was " + str(e))


#if __name__ == "__main__":
#    parser = argparse.ArgumentParser()
#    parser.add_argument("--pip", type=str, default=os.environ.get('ROBOT_HOST', 'pepper1-2'),
#                      help="Robot IP address. On robot or Local Naoqi: use '127.0.0.1'.")
#    parser.add_argument("--pport", type=int, default=9559,
#                      help="Naoqi port number")
#    parser.add_argument("--text", type=str,
#                      help="Text to say (will exit after uttering if specified)")
    
#    args = parser.parse_args()
session = qi.Session()
qilog = qi.logging.Logger("robothouse.tts")
try:
    qilog.info("Connecting to  \"" + IP + ":" + str(port) + "\"...")
    #connection_url = "tcp://" + args.pip + ":" + str(args.pport)
    app = qi.Application(["TTS", "--qi-url=" + connection_url])
    qilog.info("Connected.")
except RuntimeError:
    print ("Can't connect to Naoqi at ip \"" + IP + "\" on port " + str(port) +".\n"
          "Please check your script arguments. Run with -h option for help.")
    qilog.error("Can't connect to  \"" + IP + ":" + str(port) + "\"")
    #sys.exit(1)

tts = TTSModule(app, qilog)


In [7]:
tts.run("Hello, world!")

In [8]:
tab_pic = ALProxy("ALTabletService", IP, 9559)


In [9]:
tab_pic.session = session

In [10]:
# Ensure that the tablet wifi is enable
tab_pic.enableWifi()


In [11]:
picURL = 'http://www.manalhelal.com/files_uh/UKRob/image_1_124_P4.png'

#tab_pic.showImage("image_11_5_P10.png") # replace with an image uploaded to pepper
tab_pic.loadUrl(picURL)


False

In [12]:
tab_pic.showWebview(picURL) # this works on router network and online generated url, not the local one.


False

In [13]:
def showImage(app, picURL):
    """
    This example uses the showImage method.
    To Test ALTabletService, you need to run the script ON the robot.
    """
    # Get the service ALTabletService.
    session = app.session
    try:
        tabletService = session.service("ALTabletService")

        # Display a local image located in img folder in the root of the web server
        # The ip of the robot from the tablet is 198.18.0.1
        tabletService.enableWifi()
        tabletService.loadUrl(picURL)
        tabletService.showWebview(picURL)

        #time.sleep(3)

        # Hide the web view
        tabletService.hideImage()
    except Exception, e:
        print "Error was: ", e

In [14]:
picURL = 'http://www.manalhelal.com/files_uh/UKRob/image_1_124_P4.png'
showImage(app, picURL)

In [1]:
# Check if directory exists (in current location)
def cdTree(ftp, currentDir):
    if currentDir != "":
        try:
            ftp.cwd(currentDir)
            return True
        #except IOError:
        except :
            try:
                #cdTree(ftp, "/".join(currentDir.split("/")[:-1]))
                ftp.mkd(currentDir)
                ftp.cwd(currentDir)
                return True
            except:
                return False
def sendFile (ftp, filename, content):
    # write to local file the new content
    fhandle = open(filename, "w")
    fhandle.write(content)
    fhandle.close()
    # sent to ftp



    # local file name you want to upload
    #filename = "some_file.txt"
    with open(filename, "rb") as f:
        # use FTP's STOR command to upload the file
        ftp.storbinary("STOR %s" %filename, f)

    # list current files &amp; directories
    ftp.dir()

def sendImage (ftp, filename):
    # local file name you want to upload
    #filename = "some_file.txt"
    with open(filename, "rb") as f:
        # use FTP's STOR command to upload the file
        ftp.storbinary("STOR %s" %filename, f)

    # list current files &amp; directories
    ftp.dir()
def retrieveFile (ftp, filename):
    
    with open(filename, "wb") as f:
        # use FTP's RETR command to download the file
        ftp.retrbinary("RETR %s" %filename, f.write)
    with open(filename) as f:
        s = f.read()
        return s

In [38]:
import ftplib
session = ftplib.FTP('manalhelal.com','manalorama','Manal1974*')
#session.cwd('MANALHELAL.COM/public/files_uh/UKRob')
remotepath = 'MANALHELAL.COM/public/files_uh/UKRob'
print(cdTree (session, remotepath))
session.encoding = "utf-8"

True


In [39]:
# create new remote directory in the current remote path
#http://www.manalhelal.com/files_uh/UKRob/group_1/
remotepath = 'group_9_j3jjDNESR8yoAMCD7oEiDD'
print(cdTree (session, remotepath))

True


In [40]:
session.dir()

drwxr-xr-x   2 manalorama pg1675152       88 Jun 17 14:52 0
drwxr-xr-x   2 manalorama pg1675152       10 Jun 17 14:53 1


In [41]:
def getRemoteFiles_t (session):
    files = []

    try:
        session.dir('-t',files.append)
        newFiles = []
        for f in files:
            #print (f, i)
            tokens= f.split()
            print (tokens)
            newFiles.append (tokens[-1])
        return newFiles
    except:
        return None

In [42]:
files = getRemoteFiles_t (session)
files

['drwxr-xr-x', '2', 'manalorama', 'pg1675152', '10', 'Jun', '17', '14:53', '1']
['drwxr-xr-x', '2', 'manalorama', 'pg1675152', '88', 'Jun', '17', '14:52', '0']


['1', '0']

In [43]:
import os

def PepperProcessfiles (counter):
    if counter >= 0:
        print(cdTree (session, str(counter)))
    files = getRemoteFiles_t (session)
    if len(files) > 0:
        i=0
        f = files[0]
        slideNum = ""
        for j in range (len(f)):
            if f[j].isdigit():
                slideNum = slideNum + f[j]

        slideNum = int(slideNum)
        print (f, slideNum)
            
        if counter < 0:
            ImageNum = 0
        else:
            ImageNum = counter
        if f.endswith('.png'):
            picURL = 'http://www.manalhelal.com/files_uh/UKRob/'+remotepath+"/"+str(ImageNum)+"/"+f
            #tab_pic.showImage("image_11_5_P10.png") # replace with an image uploaded to pepper
            #tab_pic.loadUrl(picURL)
            print (picURL)        
        genTextFile = "genText_"+str(slideNum)+".txt"
        genImageText = "genImageText_"+str(slideNum)+".txt"
        genImageName = "image_"+str(slideNum)+".png"
            
        if (i+1<len(files)):
            f2 = files[i+1]
            print (f2)
            if f2.endswith('.txt'):
                text = retrieveFile (session, f2)
                #tts.run(text)
                print (text) 

        if (i+2<len(files)):
            f3 = files[i+2]
            print (f3)
            if f3.endswith('.txt'):
                text = retrieveFile (session, f3)
                #tts.run(text)
                print (text) 
    if counter >= 0:
        print(cdTree (session, "../"))


In [44]:
PepperProcessfiles(-1)

['drwxr-xr-x', '2', 'manalorama', 'pg1675152', '10', 'Jun', '17', '14:53', '1']
['drwxr-xr-x', '2', 'manalorama', 'pg1675152', '88', 'Jun', '17', '14:52', '0']
('1', 1)
0


In [47]:
i=0
PepperProcessfiles(i)

True
['-rw-r--r--', '1', 'manalorama', 'pg1675152', '3162696', 'Jun', '17', '14:52', 'image_0.png']
['-rw-r--r--', '1', 'manalorama', 'pg1675152', '522', 'Jun', '17', '14:52', 'genImageText_0.txt']
['-rw-r--r--', '1', 'manalorama', 'pg1675152', '334', 'Jun', '17', '14:52', 'genText_0.txt']
('image_0.png', 0)
http://www.manalhelal.com/files_uh/UKRob/group_9_j3jjDNESR8yoAMCD7oEiDD/0/image_0.png
genImageText_0.txt
Create an illustrated page from a children's story book. Show two sisters, Zana, an 8-years old, and Hannah, a 5-year-old, standing together in colourful superhero attire, their unique emblems proudly displayed. They are curious with bright eyes, open to learning and adventures. On the page include a cleverly designed gadget that represents generative AI, which should be kid-friendly and interesting. The setting is their bedroom which is filled with books and toys, symbolizing a nurturing and educational environment.
genText_0.txt
Zana and Hannah can start using generative AI in

In [51]:
i=i+1
PepperProcessfiles(i)

True
['-rw-r--r--', '1', 'manalorama', 'pg1675152', '3162696', 'Jun', '17', '14:53', 'image_1.png']
['-rw-r--r--', '1', 'manalorama', 'pg1675152', '597', 'Jun', '17', '14:53', 'genImageText_1.txt']
['-rw-r--r--', '1', 'manalorama', 'pg1675152', '648', 'Jun', '17', '14:53', 'genText_1.txt']
('image_1.png', 1)
http://www.manalhelal.com/files_uh/UKRob/group_9_j3jjDNESR8yoAMCD7oEiDD/1/image_1.png
genImageText_1.txt
A children's book illustration is required. The scene in focus is Xena, a young girl exhibiting her graceful karate moves. This needs to be detailed, showcasing jumps, turns, and strikes. Another character needed is Hannah, a girl of the same age, eager to explore new sports. Display a series of different sports around her; like swimming, basketball, and dance, to suggest her interest in picking up a new sport. Both girls are getting assistance from a gentle generative AI helper, shown as a friendly holographic figure offering advice. The output needs to spark curiosity and love

In [50]:
i=0

In [99]:

tts.run(retrieveFile(session, "genText_1.txt"))

'Hello World\n'

In [ ]:
session.pwd()

In [66]:
url = 'http://www.manalhelal.com/files_uh/UKRob/'
picURL =url+remotepath

In [67]:
remotepath

'group_1_VmNVktUkad34dcBKX5RWrM'

In [23]:
def PepperProcessfiles (files, newfiles):
    for f in newfiles:
        if f not in files:

            slideNum = ""
            for i in range (len(f)):
                if f[i].isdigit():
                    slideNum = slideNum + f[i]

            slideNum = int(slideNum)

            if f.endswith('.txt'):
                text = retrieveFile (session, f)
                tts.run(text)
            else:
                picURL = 'http://www.manalhelal.com/files_uh/UKRob/'+remotepath+"/"+f

                #tab_pic.showImage("image_11_5_P10.png") # replace with an image uploaded to pepper
                tab_pic.loadUrl(picURL)

In [32]:
def PepperProcessfiles (files, newfiles):
    i = 0
    while i < len(newfiles):
        f = newfiles[i]
        if f not in files:
            slideNum = ""
            for j in range (len(f)):
                if f[j].isdigit():
                    slideNum = slideNum + f[j]

            slideNum = int(slideNum)
            print (f, slideNum)
            
            
            if f.endswith('.png'):
                picURL = 'http://www.manalhelal.com/files_uh/UKRob/'+remotepath+"/"+f
                #tab_pic.showImage("image_11_5_P10.png") # replace with an image uploaded to pepper
                tab_pic.loadUrl(picURL)
                    
            genTextFile = "genText_"+str(slideNum)+".txt"
            genImageText = "genImageText_"+str(slideNum)+".txt"
            genImageName = "image_"+str(slideNum)+".png"
            
            

            if (i+1<len(newfiles)):
                f2 = newfiles[i+1]
                print (f2)
                if f2.endswith('.txt'):
                    text = retrieveFile (session, f2)
                    tts.run(text)
                
            if (i+2<len(newfiles)):
                f3 = newfiles[i+2]
                print (f3)
                if f3.endswith('.txt'):
                    text = retrieveFile (session, f3)
                    tts.run(text)
        i = i+3

In [33]:
#files = getRemoteFiles_t (session)
PepperProcessfiles ([], files)


('image_0.png', 0)
genImageText_0.txt
genText_0.txt


In [34]:
newfiles = getRemoteFiles_t (session)
PepperProcessfiles (files, newfiles)

['-rw-r--r--', '1', 'manalorama', 'pg1675152', '3162696', 'Jun', '17', '08:29', 'image_0.png']
['-rw-r--r--', '1', 'manalorama', 'pg1675152', '528', 'Jun', '17', '08:28', 'genImageText_0.txt']
['-rw-r--r--', '1', 'manalorama', 'pg1675152', '363', 'Jun', '17', '08:28', 'genText_0.txt']


In [15]:
with open('pres.txt') as f:
    lines = [line.rstrip('\n') for line in f]

In [16]:
for l in lines:
    tts.run(l)
    print (l)

Hello

I hope you enjoyed your activity and generated good stories. 

Please ensure you filled out your Questionnaires at the start and end of this activity.

Let's now try to learn how AI generated these stories for you.

We need to learn the limitations of AI and how to use it ethically and efficiently.

Neural Networks are so simple that they can be explained to babies, as done in this book by Chris Ferrie and Dr Sarah Kaiser

Let's see some pages from this book:

Your eye (sensor) passes to the brain what you see. This activates a neuron.

For example, when you see a red ball, your eye is activated, and this ball is passed as input to other neurons.

Every neuron tries to understand and keep activating other neurons (maybe a memory of other images you saw before) to learn the image's description.

One neuron can be activated by one input or more. The neuron outputs one or more outputs to activate another one more neuron. Maybe you saw a lion and must activate your legs to escape.

